# Semi Deviation, VAR and CVAR


What Is Downside? Downside is the negative movement in the price of a security, sector or market. 

# 0. Downside

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats

def drawdown(return_series: pd.Series):
    """Takes a time series of asset returns.
       returns a DataFrame with columns for
       the wealth index, 
       the previous peaks, and 
       the percentage drawdown
    """
    wealth_index = 1000*(1+return_series).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks
    return pd.DataFrame({"Wealth": wealth_index, 
                         "Previous Peak": previous_peaks, 
                         "Drawdown": drawdowns})


def get_ffme_returns():
    """
    Load the Fama-French Dataset for the returns of the Top and Bottom Deciles by MarketCap
    """
    me_m = pd.read_csv("../input/edhec-data-for-portfolio-construction-with-python/Portfolios_Formed_on_ME_monthly_EW.csv",
                       header=0, index_col=0, na_values=-99.99)
    rets = me_m[['Lo 10', 'Hi 10']]
    rets.columns = ['SmallCap', 'LargeCap']
    rets = rets/100
    rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')
    return rets


def get_hfi_returns():
    """
    Load and format the EDHEC Hedge Fund Index Returns
    """
    hfi = pd.read_csv("../input/edhec-data-for-portfolio-construction-with-python/edhec-hedgefundindices.csv",
                      header=0, index_col=0, parse_dates=True)
    hfi = hfi/100
    hfi.index = hfi.index.to_period('M')
    return hfi

def skewness(r):
    """
    Alternative to scipy.stats.skew()
    Computes the skewness of the supplied Series or DataFrame
    Returns a float or a Series
    """
    return scipy.stats.skew(r)



def kurtosis(r):
    """
    Alternative to scipy.stats.kurtosis()
    Computes the kurtosis of the supplied Series or DataFrame
    Returns a float or a Series
    """
    return scipy.stats.kurtosis(r)

In [ ]:
hfi=get_hfi_returns()
hfi.head()

# 1. Semideviation

In [ ]:
# standard deviation
hfi.std()

In [ ]:
# however we only compute the deviation towards the downside
hfi[hfi<0].std()

In [ ]:
# make it a function
def semideviation(r):
    '''
    returns the semideviation aka negative semideviation of r
    r must be a Series or a DataFrame
    '''
    return r[r<0].std(ddof=0)

In [ ]:
semideviation(hfi)

# 2. VaR

* 2.1 historical VaR
* 2.2 Gaussian Parametric VaR
* 2.3 Modified Cornish-Fisher VaR

## 2.1 historical VaR

**Historical value at risk (VaR), also known as historical simulation or the historical method, refers to a particular way of calculating VaR. In this approach we calculate VaR directly from past returns. For example, suppose we want to calculate the 1-day 95% VaR for an equity using 100 days of data. The 95th percentile corresponds to the least worst of the worst 5% of returns. In this case, because we are using 100 days of data, the VaR simply corresponds to the 5th worst day.**
https://corporatefinanceinstitute.com/resources/knowledge/trading-investing/value-at-risk-var/

In [ ]:
print(np.percentile(hfi,5,axis=0))
print(hfi.columns)

In [ ]:
def var_historic(r, level=5):
    '''
    VaR Historic
    '''
    # if r is a DataFrame , it will return a True
    # Aggregate using one or more operations over the specified axis.
    if isinstance(r,pd.DataFrame):
        return r.aggregate(var_historic,level=level) # call on each columns
    elif isinstance (r,pd.Series):
        return -np.percentile(r,level)
    else:
        raise TypeError ('Expected r to be series or dataframe')

In [ ]:
var_historic(hfi)

## 2.2 Gaussian Pasrametric VaR (standard normal gaussian)

**VaR = - (mean + z * std)**

In [ ]:
from scipy.stats import norm
# get the Z-score

![](https://www.investopedia.com/thmb/qnWc7_zqmjH4rB0ft3K_IRSIslE=/1787x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/Variance-CovarianceMethod5-5bde86ce7819405ca63f26aa275a4bd2.png)
https://www.investopedia.com/articles/04/092904.asp

In [ ]:
# get the Z-score 
norm.ppf(0.95)

In [ ]:
# calculate the Z-score for the 5%, and calculate the VaR using
# VaR = - (mean + z * std)
z=norm.ppf(0.05)
-(hfi.mean()+z*hfi.std(ddof=0))

In [ ]:
# make it a function
def var_gaussian(r, level=5):
    z=norm.ppf(level/100)
    return -(r.mean()+z*r.std(ddof=0))

In [ ]:
var_gaussian(hfi)

## 2.3 Cornish-Fisher VaR

![](https://image.slidesharecdn.com/infiniticapitalfourmomentriskdecomposition-111128005125-phpapp02/95/four-moment-risk-decomposition-presentation-6-728.jpg?cb=1322442187)

In [ ]:
# key: modified z from the: gaussian z, skewness, kurtosis

# make it a function
def var_gaussian(r, level=5,modified=False):
    z=norm.ppf(level/100)
    # the Cornish Fisher option
    if modified:
        # modified z from s, k and z
        s=skewness(r)
        k=kurtosis(r)
        z=(z+
           (z**2-1)*s/6+
           (z**3-3*z)*(k-3)/24-
           (2*z**3-5*z)*(s**2)/36
        )
    
    return -(r.mean()+z*r.std(ddof=0))


In [ ]:
var_list=(var_gaussian(hfi),var_gaussian(hfi,modified=True),var_historic(hfi))
comparison=pd.concat(var_list,axis=1)
comparison.columns=['Gaussian','Cornish-Fisher','Historic']
comparison

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize = (10, 5))
comparison.plot.bar(title='Hedge Fund Indices: VaR')

most cases the Cornish-Fisher produces higher VaR

# 3. CVaR (BeyondVaR)

**Definition**: CVaR is derived by taking a weighted average of the “extreme” losses in the tail of the distribution of possible returns, beyond the value at risk (VaR) cutoff point. 
URL: https://www.investopedia.com/terms/c/conditional_value_at_risk.asp
![](https://www.researchgate.net/profile/Y-Vardanyan/publication/322917958/figure/fig6/AS:589989427560454@1517675849921/Demonstration-of-the-VaR-and-CVaR-concepts.png)

In [ ]:
# CVaR: average of all the returns that worse(beyond) the VaR
def cvar_historic(r, level=5):
    if isinstance(r,pd.Series):
        below_var_True=r<=-var_historic(r,level=level) # find all the returns that less thant the VaR, which gives a mask
        return -r[below_var_True].mean()
    elif isinstance (r,pd.DataFrame):
        return r.aggregate(cvar_historic,level=level)
    else:
        raise TypeError ('Expected r to be a Series or DataFrame')

In [ ]:
# if the worst sitiuation with 5% chance occured, the average loss (expected loss) would be in 1 month:
cvar_historic(hfi)

# Some other exploration

In [ ]:
hfi=hfi['2000':'2018']

In [ ]:
kurtosis(hfi)

In [ ]:
hfi.columns